In [1]:
# pulls from the FantasyPros and ESPN API

## **Important: The dataframes are built using csv files in the current working directory so 
# do not delete or comment out these functions or lines of code that create the csv files

In [2]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime
import nfl_data_py as nfl
import os

In [3]:
# Define the template URLs for the QB, RB, and WR positions (weekly stats)
espn_urls = {
    "QB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Apassing&sort=passing.passingYards%3Adesc&season={year}&seasontype={seasontype}",
    "RB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Arushing&sort=rushing.rushingYards%3Adesc&season={year}&seasontype={seasontype}",
    "WR": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Areceiving&sort=receiving.receivingYards%3Adesc&season={year}&seasontype={seasontype}"
}

In [4]:
# **IMPORTANT: this function outputs the combined qb-betting lines df
# Get current year and week for NFL
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4) ## *** Reset the date at the start of the NFL season ***
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Set the current NFL year and week
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [5]:
# Adjust the fetch function to return both data and pagination info for verification
def fetch_position_data_with_verification(position, url_template):
    page = 1
    all_players = []
    total_pages = 1  # Default to 1 page unless pagination indicates more
    
    while True:
        # Construct the API URL for the current page
        url = url_template.format(page=page, year=current_year, seasontype=seasontype)
        response = requests.get(url)
        
        # If response is successful, process the data
        if response.status_code == 200:
            data = response.json()
            athletes = data.get('athletes', [])
            
            # Get pagination information for verification
            if page == 1:
                pagination = data.get('pagination', {})
                total_pages = pagination.get('pages', 1)  # Total number of pages
            
            if not athletes:
                break  # Stop if no more athletes are available
            
            for athlete_data in athletes:
                athlete = athlete_data['athlete']

                # Extract relevant data for the base columns (excluding stats)
                player_info = {
                    'year': current_year,
                    'week': current_week,
                    'player_id': athlete.get('id', 'N/A'),
                    'player': athlete.get('displayName', 'N/A'),
                    'position': position,
                    'team': athlete.get('teamShortName', 'N/A')
                }

                all_players.append(player_info)
            
            page += 1  # Increment to the next page
        else:
            break  # Stop if there's an error in fetching data
    
    return all_players, total_pages

In [6]:
# Function to convert fetched data into a DataFrame
def create_dataframe(position_data):
    return pd.DataFrame(position_data)
    
# Updated process function to include verification step
def process_and_verify_position_data():
    # Fetch data for each position and track total pages
    qb_data, qb_pages = fetch_position_data_with_verification("QB", espn_urls["QB"])
    rb_data, rb_pages = fetch_position_data_with_verification("RB", espn_urls["RB"])
    wr_data, wr_pages = fetch_position_data_with_verification("WR", espn_urls["WR"])
    
    # Convert fetched data into DataFrames
    df_qb = create_dataframe(qb_data)
    df_rb = create_dataframe(rb_data)
    df_wr = create_dataframe(wr_data)
    
    # Verification output
    print(f"QB: Fetched {len(df_qb)} rows across {qb_pages} pages.")
    print(f"RB: Fetched {len(df_rb)} rows across {rb_pages} pages.")
    print(f"WR: Fetched {len(df_wr)} rows across {wr_pages} pages.")
    
    # Display the first few rows for review
    display(df_qb.head())
    display(df_rb.head())
    display(df_wr.head())
    
    return df_qb, df_rb, df_wr

# Call the function to fetch, verify, and display the data
df_qb, df_rb, df_wr = process_and_verify_position_data()

QB: Fetched 71 rows across 2 pages.
RB: Fetched 247 rows across 5 pages.
WR: Fetched 398 rows across 8 pages.


,year,week,player_id,player,position,team
0,2024,7,15864,Geno Smith,QB,SEA
1,2024,7,4361741,Brock Purdy,QB,SF
2,2024,7,14880,Kirk Cousins,QB,ATL
3,2024,7,3915511,Joe Burrow,QB,CIN
4,2024,7,8439,Aaron Rodgers,QB,NYJ


,year,week,player_id,player,position,team
0,2024,7,3043078,Derrick Henry,RB,BAL
1,2024,7,4360569,Jordan Mason,RB,SF
2,2024,7,3929630,Saquon Barkley,RB,PHI
3,2024,7,4047365,Josh Jacobs,RB,GB
4,2024,7,4241416,Chuba Hubbard,RB,CAR


,year,week,player_id,player,position,team
0,2024,7,4362628,Ja'Marr Chase,WR,CIN
1,2024,7,4047650,DK Metcalf,WR,SEA
2,2024,7,4258173,Nico Collins,WR,HOU
3,2024,7,4262921,Justin Jefferson,WR,MIN
4,2024,7,4432773,Brian Thomas Jr.,WR,JAX


In [7]:
# Function to generate FantasyPros URLs based on the positions
# Function to generate FantasyPros URLs based on the positions
def generate_fantasy_pros_urls(season, positions=None, week=None, scoring=None):
    base_url = f"https://api.fantasypros.com/public/v2/json/nfl/{season}/projections"
    # If positions is not provided, default to QB, RB, WR. Otherwise, use the list directly.
    positions_list = ['QB', 'RB', 'WR'] if positions is None else positions  # Remove split
    scoring_str = scoring.replace("'", "") if scoring else None
    generated_urls = []

    for position in positions_list:
        params = {'position': position}
        if season:
            params['season'] = season
        if week:
            params['week'] = week
        if scoring:
            params['scoring'] = scoring_str
        query_string = requests.compat.urlencode(params)
        full_url = f"{base_url}?{query_string}"
        generated_urls.append(full_url)

    return generated_urls

# Function to fetch data from FantasyPros API
def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    try:
        response.raise_for_status()
        return response.json()  # Return the JSON data
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}. Error: {e}")
        return None

# Function to fetch and handle FantasyPros data for given positions and stats
def fetch_fantasy_pros_data(season, positions=None, week=None, scoring=None):
    api_key = os.getenv('api_key')
    if not api_key:
        print("API key is not set.")
        return None
    
    headers = {'x-api-key': api_key}
    urls = generate_fantasy_pros_urls(season, positions, week, scoring)
    all_data = []
    
    for url in urls:
        print(f"Fetching FantasyPros data from: {url}")
        response = fetch_data(url, headers)
        if response and 'players' in response:
            players_data = response['players']
            for player in players_data:
                # Extract general columns
                player_info = {
                    'name': player['name'],
                    'points': player['stats'].get('points', 0),
                    'points_ppr': player['stats'].get('points_ppr', 0),
                    'points_half': player['stats'].get('points_half', 0)
                }
                # Extract position-specific columns based on position
                position = player.get('position_id')
                if position == 'QB':
                    player_info.update({
                        'passing_attempts': player['stats'].get('pass_att', 0),
                        'passing_completions': player['stats'].get('pass_cmp', 0),
                        'passing_yards': player['stats'].get('pass_yds', 0),
                        'passing_tds': player['stats'].get('pass_tds', 0)
                    })
                elif position == 'RB':
                    player_info.update({
                        'rushing_attempts': player['stats'].get('rush_att', 0),
                        'rushing_yards': player['stats'].get('rush_yds', 0),
                        'rushing_tds': player['stats'].get('rush_tds', 0),
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                elif position == 'WR':
                    player_info.update({
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                all_data.append(player_info)
    
    return pd.DataFrame(all_data)

In [8]:
# Function to merge ESPN data with FantasyPros data, keeping only the relevant columns for each position
def merge_espn_fantasypros(espn_df, fantasypros_df, position):
    # Extract relevant columns based on position
    if position == 'QB':
        # Extract only QB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'passing_attempts', 'passing_completions', 'passing_yards', 'passing_tds']]
    elif position == 'RB':
        # Extract only RB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'rushing_attempts', 'rushing_yards', 'rushing_tds', 
                                         'receptions', 'reception_yards', 'reception_tds']]
    elif position == 'WR':
        # Extract only WR relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'receptions', 'reception_yards', 'reception_tds']]

    # Merge on 'player' from ESPN and 'name' from FantasyPros
    merged_df = pd.merge(espn_df, fantasypros_df, left_on='player', right_on='name', how='left')
    
    # Drop the redundant 'name' column from FantasyPros
    merged_df.drop(columns=['name'], inplace=True)
    
    return merged_df

In [9]:
# Function to fetch, merge, and save ESPN and FantasyPros data for all positions
def process_and_merge_fantasypros_data(df_qb, df_rb, df_wr, scoring='STD'):
    # Fetch current season and week dynamically
    current_week = get_current_week()
    season = datetime.now().year

    # Fetch FantasyPros data for all positions
    fantasypros_data = fetch_fantasy_pros_data(season=season, positions=['QB', 'RB', 'WR'], week=current_week, scoring=scoring)

    # Merging ESPN dataframes with FantasyPros data, keeping only relevant columns
    df_qb_merged = merge_espn_fantasypros(df_qb, fantasypros_data, 'QB')
    df_rb_merged = merge_espn_fantasypros(df_rb, fantasypros_data, 'RB')
    df_wr_merged = merge_espn_fantasypros(df_wr, fantasypros_data, 'WR')


    # Display the merged dataframes
    display(df_qb_merged.head())
    display(df_rb_merged.head())
    display(df_wr_merged.head())
    
    return df_qb_merged, df_rb_merged, df_wr_merged

# Call the function to fetch, merge, save, and display the data
df_qb_merged, df_rb_merged, df_wr_merged = process_and_merge_fantasypros_data(df_qb, df_rb, df_wr)


Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=QB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=RB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=WR&season=2024&week=7&scoring=STD


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds
0,2024,7,15864,Geno Smith,QB,SEA,17.68,17.68,17.68,37.83,26.32,264.92,1.45
1,2024,7,4361741,Brock Purdy,QB,SF,17.96,17.96,17.96,32.92,21.43,258.10,1.66
2,2024,7,14880,Kirk Cousins,QB,ATL,16.80,16.80,16.80,34.10,22.72,260.14,1.72
3,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.05,22.88,256.16,1.68
4,2024,7,8439,Aaron Rodgers,QB,NYJ,15.68,15.68,15.68,33.27,21.87,236.08,1.53


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds
0,2024,7,3043078,Derrick Henry,RB,BAL,15.72,16.95,16.33,19.31,91.50,0.90,1.23,10.05,0.05
1,2024,7,4360569,Jordan Mason,RB,SF,11.21,12.81,12.01,16.26,69.60,0.50,1.60,11.45,0.04
2,2024,7,3929630,Saquon Barkley,RB,PHI,14.55,17.34,15.95,19.15,85.76,0.56,2.79,20.29,0.14
3,2024,7,4047365,Josh Jacobs,RB,GB,11.02,13.46,12.24,15.54,65.24,0.40,2.44,16.89,0.09
4,2024,7,4241416,Chuba Hubbard,RB,CAR,12.46,15.74,14.10,16.59,72.52,0.40,3.28,23.83,0.10


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds
0,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.50,17.54,14.52,6.04,82.50,0.56
1,2024,7,4047650,DK Metcalf,WR,SEA,9.94,15.22,12.58,5.28,72.42,0.47
2,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,7,4262921,Justin Jefferson,WR,MIN,12.82,19.54,16.18,6.72,92.52,0.61
4,2024,7,4432773,Brian Thomas Jr.,WR,JAX,8.14,12.54,10.34,4.40,58.26,0.39


In [10]:
def scrape_salary_changes():
    # URL of the FantasyPros salary changes page
    url = "https://www.fantasypros.com/daily-fantasy/nfl/fanduel-salary-changes.php"
    
    # Fetch the page content
    response = requests.get(url)
    
    # Check if the page was fetched successfully
    if response.status_code != 200:
        print(f"Failed to fetch the page. Status code: {response.status_code}")
        return
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Locate the table containing the salary changes (assuming it's the first table)
    table = soup.find('table')  # Adjust if necessary based on the page structure
    
    # Extract the table headers
    headers = [header.text for header in table.find_all('th')]
    
    # Extract the table rows
    rows = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]  # Clean up whitespace
        rows.append(cols)
    
    # Create a DataFrame with the scraped data
    salary_changes_df = pd.DataFrame(rows, columns=headers)
        
    # Display the first few rows of the DataFrame
    display(salary_changes_df.head())
    
    return salary_changes_df

# Run the function to scrape and display salary changes
scrape_salary_changes()

,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,245,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,386,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,245,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,386,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400
...,...,...,...,...,...,...,...
961,-,D.J. Montgomery (IND - WR),Sun 1:00PM,MIA,"$4,000","$4,000",0
962,-,Montrell Washington (KC - WR),Sun 4:25PM,@SF,"$4,000",-,-
963,-,Nick Muse (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
964,-,Owen Wright (BAL - RB),Mon 8:15PM,@TB,"$4,000","$4,000",0


In [11]:
def fetch_salary_changes():
    # Directly scrape and return the salary changes DataFrame
    print("Fetching salary changes data by scraping...")
    salary_changes_df = scrape_salary_changes()
    return salary_changes_df

In [12]:
def process_salary_changes_by_position():
    # Step 1: Fetch the salary changes data in memory
    salary_changes_df = fetch_salary_changes()
    
    # Step 2: Split 'Player' column into three separate columns (FirstName, LastName, Team-Position)
    salary_changes_df[['FirstName', 'LastName', 'Team-Position']] = salary_changes_df['Player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
    
    # Step 3: Drop the original 'Player' column and the first column (Rankings)
    salary_changes_df.drop(columns=['Player', salary_changes_df.columns[0]], inplace=True)
    
    # Step 4: Split 'Team-Position' into separate 'Team' and 'Position' columns
    salary_changes_df[['Team', 'Position']] = salary_changes_df['Team-Position'].str.extract(r'(\w+)\s*-\s*(\w+)')
    
    # Step 5: Drop 'Team-Position' and any unwanted columns like 'Kickoff' and 'Opp'
    salary_changes_df.drop(columns=['Team-Position', 'Kickoff', 'Opp'], inplace=True, errors='ignore')
    
    # Step 6: Reorder the columns to match the desired order
    salary_changes_df = salary_changes_df[['FirstName', 'LastName', 'Team', 'Position', 'This Week', 'Last Week', 'Difference']]
    
    # Step 7: Split the DataFrame by position
    df_qb = salary_changes_df[salary_changes_df['Position'] == 'QB']
    df_wr = salary_changes_df[salary_changes_df['Position'] == 'WR']
    df_rb = salary_changes_df[salary_changes_df['Position'] == 'RB']
    
    # Display the first few rows of each DataFrame for verification
    display(df_qb.head())
    display(df_wr.head())
    display(df_rb.head())
    
    return df_qb, df_wr, df_rb

# Call the function to process the salary changes by position
df_qb_salary, df_wr_salary, df_rb_salary = process_salary_changes_by_position()


Fetching salary changes data by scraping...


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,245,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,386,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
4,Jayden,Daniels,WAS,QB,"$9,100","$8,700",+400
13,Kedon,Slovis,HOU,QB,"$6,000","$6,000",0
15,Bo,Nix,DEN,QB,"$6,800","$6,700",+100
21,Max,Duggan,LAC,QB,"$6,000","$6,000",0
41,Easton,Stick,LAC,QB,"$6,000","$6,000",0


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
1,Metchie,III,HOU,WR,"$4,400","$4,300",+100
5,Ricky,Pearsall,SF,WR,"$4,800","$4,000",+800
7,Joseph,Ngata,PHI,WR,"$4,000","$4,000",0
8,Anthony,Gould,IND,WR,"$4,000","$4,000",0
10,Trey,Palmer,TB,WR,"$4,200","$4,200",0


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
3,Kevin,Harris,NE,RB,"$4,000","$4,000",0
12,Eric,Gray,NYG,RB,"$5,000","$4,700",+300
16,Austin,Jones,WAS,RB,"$4,000","$4,000",0
19,Breece,Hall,NYJ,RB,"$7,600","$8,000",-400
25,Aidan,Robbins,CLE,RB,"$4,000","$4,000",0


In [13]:
# merge salary and projections
def merge_salary_and_projections(df_qb_salary, df_rb_salary, df_wr_salary, df_qb_proj, df_rb_proj, df_wr_proj):
    """
    This function takes the salary DataFrames and the projection DataFrames 
    and merges them based on player names for QB, RB, and WR positions.
    """

    # Split 'player' column into 'FirstName' and 'LastName' in projection DataFrames
    df_qb_proj[['FirstName', 'LastName']] = df_qb_proj['player'].str.split(' ', n=1, expand=True)
    df_rb_proj[['FirstName', 'LastName']] = df_rb_proj['player'].str.split(' ', n=1, expand=True)
    df_wr_proj[['FirstName', 'LastName']] = df_wr_proj['player'].str.split(' ', n=1, expand=True)

    # Merge salary and projections for QB
    df_qb_merge_projection_salary = pd.merge(df_qb_proj, df_qb_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"QB merged data: {df_qb_merge_projection_salary.shape[0]} rows")

    # Merge salary and projections for RB
    df_rb_merge_projection_salary = pd.merge(df_rb_proj, df_rb_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"RB merged data: {df_rb_merge_projection_salary.shape[0]} rows")

    # Merge salary and projections for WR
    df_wr_merge_projection_salary = pd.merge(df_wr_proj, df_wr_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"WR merged data: {df_wr_merge_projection_salary.shape[0]} rows")

    # Display the first few rows of each merged DataFrame for verification
    display(df_qb_merge_projection_salary.head())
    display(df_rb_merge_projection_salary.head())
    display(df_wr_merge_projection_salary.head())

    return df_qb_merge_projection_salary, df_rb_merge_projection_salary, df_wr_merge_projection_salary


# Fetch, merge, and return projections from ESPN and FantasyPros (already implemented)
df_qb_proj, df_rb_proj, df_wr_proj = process_and_merge_fantasypros_data(df_qb, df_rb, df_wr, scoring='STD')

# Now, merge these with salary data
df_qb_merge_projection_salary, df_rb_merge_projection_salary, df_wr_merge_projection_salary = merge_salary_and_projections(
    df_qb_salary, df_rb_salary, df_wr_salary,  # Salary DataFrames
    df_qb_proj, df_rb_proj, df_wr_proj  # Projection DataFrames from ESPN and FantasyPros
)


Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=QB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=RB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=WR&season=2024&week=7&scoring=STD


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds
0,2024,7,15864,Geno Smith,QB,SEA,17.68,17.68,17.68,37.83,26.32,264.92,1.45
1,2024,7,4361741,Brock Purdy,QB,SF,17.96,17.96,17.96,32.92,21.43,258.10,1.66
2,2024,7,14880,Kirk Cousins,QB,ATL,16.80,16.80,16.80,34.10,22.72,260.14,1.72
3,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.05,22.88,256.16,1.68
4,2024,7,8439,Aaron Rodgers,QB,NYJ,15.68,15.68,15.68,33.27,21.87,236.08,1.53


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds
0,2024,7,3043078,Derrick Henry,RB,BAL,15.72,16.95,16.33,19.31,91.50,0.90,1.23,10.05,0.05
1,2024,7,4360569,Jordan Mason,RB,SF,11.21,12.81,12.01,16.26,69.60,0.50,1.60,11.45,0.04
2,2024,7,3929630,Saquon Barkley,RB,PHI,14.55,17.34,15.95,19.15,85.76,0.56,2.79,20.29,0.14
3,2024,7,4047365,Josh Jacobs,RB,GB,11.02,13.46,12.24,15.54,65.24,0.40,2.44,16.89,0.09
4,2024,7,4241416,Chuba Hubbard,RB,CAR,12.46,15.74,14.10,16.59,72.52,0.40,3.28,23.83,0.10


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds
0,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.50,17.54,14.52,6.04,82.50,0.56
1,2024,7,4047650,DK Metcalf,WR,SEA,9.94,15.22,12.58,5.28,72.42,0.47
2,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,7,4262921,Justin Jefferson,WR,MIN,12.82,19.54,16.18,6.72,92.52,0.61
4,2024,7,4432773,Brian Thomas Jr.,WR,JAX,8.14,12.54,10.34,4.40,58.26,0.39


QB merged data: 72 rows
RB merged data: 247 rows
WR merged data: 398 rows


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,15864,Geno Smith,QB,SEA,17.68,17.68,17.68,37.83,26.32,264.92,1.45,Geno,Smith,SEA,QB,"$7,600","$7,900",-300
1,2024,7,4361741,Brock Purdy,QB,SF,17.96,17.96,17.96,32.92,21.43,258.10,1.66,Brock,Purdy,SF,QB,"$7,700","$7,800",-100
2,2024,7,14880,Kirk Cousins,QB,ATL,16.80,16.80,16.80,34.10,22.72,260.14,1.72,Kirk,Cousins,ATL,QB,"$7,200","$7,400",-200
3,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.05,22.88,256.16,1.68,Joe,Burrow,CIN,QB,"$8,200","$8,100",+100
4,2024,7,8439,Aaron Rodgers,QB,NYJ,15.68,15.68,15.68,33.27,21.87,236.08,1.53,Aaron,Rodgers,NYJ,QB,"$6,900","$7,200",-300


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,...,receptions,reception_yards,reception_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,3043078,Derrick Henry,RB,BAL,15.72,16.95,16.33,19.31,...,1.23,10.05,0.05,Derrick,Henry,BAL,RB,"$9,100","$9,100",0
1,2024,7,4360569,Jordan Mason,RB,SF,11.21,12.81,12.01,16.26,...,1.60,11.45,0.04,Jordan,Mason,SF,RB,"$8,800","$8,900",-100
2,2024,7,3929630,Saquon Barkley,RB,PHI,14.55,17.34,15.95,19.15,...,2.79,20.29,0.14,Saquon,Barkley,PHI,RB,"$9,000","$9,200",-200
3,2024,7,4047365,Josh Jacobs,RB,GB,11.02,13.46,12.24,15.54,...,2.44,16.89,0.09,Josh,Jacobs,GB,RB,"$7,300","$7,600",-300
4,2024,7,4241416,Chuba Hubbard,RB,CAR,12.46,15.74,14.10,16.59,...,3.28,23.83,0.10,Chuba,Hubbard,CAR,RB,"$7,400","$7,400",0


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.50,17.54,14.52,6.04,82.50,0.56,Ja'Marr,Chase,NaN,NaN,NaN,NaN,NaN
1,2024,7,4047650,DK Metcalf,WR,SEA,9.94,15.22,12.58,5.28,72.42,0.47,DK,Metcalf,SEA,WR,"$8,100","$8,300",-200
2,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN,Nico,Collins,HOU,WR,"$4,000","$8,900","-4,900"
3,2024,7,4262921,Justin Jefferson,WR,MIN,12.82,19.54,16.18,6.72,92.52,0.61,Justin,Jefferson,MIN,WR,"$9,400",-,-
4,2024,7,4432773,Brian Thomas Jr.,WR,JAX,8.14,12.54,10.34,4.40,58.26,0.39,Brian,Thomas Jr.,NaN,NaN,NaN,NaN,NaN


In [14]:
# Modify Columns & Merge Salary/Projections
def modify_cols_merge_salary_and_projections(projections_df, salary_df):
    """
    This function merges salary and projection data based on FirstName and LastName,
    and reorders columns accordingly while removing unnecessary columns.
    """
    # Ensure salary_df has FirstName and LastName already
    if 'FirstName' not in salary_df.columns or 'LastName' not in salary_df.columns:
        print("Error: 'FirstName' and/or 'LastName' columns are missing in salary DataFrame.")
        print(f"Available columns in salary_df: {salary_df.columns}")
        return None
    
    # Ensure projections have player names split into firstName and lastName
    projections_df[['firstName', 'lastName']] = projections_df['player'].str.split(' ', n=1, expand=True)

    # Debugging: Print columns to ensure they exist
    print("Projections DataFrame columns:", projections_df.columns)
    print("Salary DataFrame columns:", salary_df.columns)
    
    # Merge the two dataframes
    merged_df = pd.merge(projections_df, salary_df, left_on=['firstName', 'lastName'], right_on=['FirstName', 'LastName'], how='left')

    # Drop unwanted columns
    columns_to_drop = ['FirstName_x', 'LastName_x', 'FirstName_y', 'LastName_y', 'Team', 'Position']
    existing_columns_to_drop = [col for col in columns_to_drop if col in merged_df.columns]  # Only drop if they exist
    merged_df.drop(columns=existing_columns_to_drop, inplace=True)

    # Reorder columns to move 'firstName' before 'lastName' right after 'player'
    cols = list(merged_df.columns)
    player_index = cols.index('player')
    first_last_name = ['firstName', 'lastName']
    
    for name in first_last_name[::-1]:  # Reverse the order to insert correctly
        cols.insert(player_index + 1, cols.pop(cols.index(name)))
    
    merged_df = merged_df[cols]
    
    return merged_df

# Merge Salary and Projections
def merge2_salary_and_projections(df_qb_proj, df_rb_proj, df_wr_proj, df_qb_salary, df_rb_salary, df_wr_salary):
    """
    Merges salary and projections data for QB, RB, and WR positions.
    """
    # Merge the projections and salary changes for QB
    qb_projections_salary_merged_df = modify_cols_merge_salary_and_projections(df_qb_proj, df_qb_salary)
    print(f"QB merged data: {qb_projections_salary_merged_df.shape[0]} rows")
    
    # Merge the projections and salary changes for RB
    rb_projections_salary_merged_df = modify_cols_merge_salary_and_projections(df_rb_proj, df_rb_salary)
    print(f"RB merged data: {rb_projections_salary_merged_df.shape[0]} rows")
    
    # Merge the projections and salary changes for WR
    wr_projections_salary_merged_df = modify_cols_merge_salary_and_projections(df_wr_proj, df_wr_salary)
    print(f"WR merged data: {wr_projections_salary_merged_df.shape[0]} rows")
    
    return qb_projections_salary_merged_df, rb_projections_salary_merged_df, wr_projections_salary_merged_df


# Call the function with in-memory DataFrames
df_qb_merge2_projection_salary, df_rb_merge2_projection_salary, df_wr_merge2_projection_salary = merge2_salary_and_projections(
    df_qb_proj, df_rb_proj, df_wr_proj,  # Projections DataFrames
    df_qb_salary, df_rb_salary, df_wr_salary  # Salary DataFrames
)

Projections DataFrame columns: Index(['year', 'week', 'player_id', 'player', 'position', 'team', 'points',
       'points_ppr', 'points_half', 'passing_attempts', 'passing_completions',
       'passing_yards', 'passing_tds', 'FirstName', 'LastName', 'firstName',
       'lastName'],
      dtype='object')
Salary DataFrame columns: Index(['FirstName', 'LastName', 'Team', 'Position', 'This Week', 'Last Week',
       'Difference'],
      dtype='object')
QB merged data: 72 rows
Projections DataFrame columns: Index(['year', 'week', 'player_id', 'player', 'position', 'team', 'points',
       'points_ppr', 'points_half', 'rushing_attempts', 'rushing_yards',
       'rushing_tds', 'receptions', 'reception_yards', 'reception_tds',
       'FirstName', 'LastName', 'firstName', 'lastName'],
      dtype='object')
Salary DataFrame columns: Index(['FirstName', 'LastName', 'Team', 'Position', 'This Week', 'Last Week',
       'Difference'],
      dtype='object')
RB merged data: 247 rows
Projections DataFr

In [15]:
def scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week'):
    current_week = get_current_week()
    
    # Dictionary to store DataFrames for each position
    rostered_dataframes = {}
    
    for position in positions:
        # Initialize an empty DataFrame to accumulate data
        rostered_df = pd.DataFrame()
        
        # Loop over weeks from current_week down to 1
        for week in range(current_week, 0, -1):
            # Construct the URL for the given position and week
            url = f"https://www.fantasypros.com/nfl/stats/{position}.php?week={week}&scoring={scoring}&range={range_type}"
            
            # Fetch the page content
            response = requests.get(url)
            
            # Check if the page was fetched successfully
            if response.status_code != 200:
                print(f"Failed to fetch the page for {position} week {week}. Status code: {response.status_code}")
                continue
            
            # Parse the page content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Locate the table containing the stats data
            table = soup.find('table')
            
            # Extract the table headers
            headers = [header.text.strip() for header in table.find_all('th')]
            
            # Find the indices of the "Player" and "ROST" columns
            if "Player" not in headers or "ROST" not in headers:
                print(f"'Player' or 'ROST' column not found for {position} week {week}.")
                continue
            
            player_index = headers.index("Player")
            rost_index = headers.index("ROST")
            
            # Extract the table rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip the header row
                cols = row.find_all('td')
                cols = [ele.get_text(strip=True) for ele in cols]  # Clean up whitespace
                
                # Only process rows with the expected number of columns
                if len(cols) >= rost_index + 1:
                    player_name = cols[player_index]  # Player name
                    rost_percentage = cols[rost_index]  # Rostered percentage
                    rows.append([player_name, rost_percentage])
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=["Player", f"wk{week}"])
            
            # Convert the rostered percentage to float (remove '%' sign)
            week_df[f"wk{week}"] = week_df[f"wk{week}"].str.replace('%', '').astype(float)
            
            if rostered_df.empty:
                # First week's data, initialize the cumulative DataFrame
                rostered_df = week_df
            else:
                # Merge with the cumulative DataFrame
                rostered_df = pd.merge(rostered_df, week_df, on='Player', how='outer')
        
        # After looping through all weeks, sort columns in descending order
        # First, get the list of columns except 'Player'
        cols = rostered_df.columns.tolist()
        cols.remove('Player')
        # Sort the week columns in descending order
        cols_sorted = sorted(cols, key=lambda x: int(x[2:]), reverse=True)
        # Rearrange the columns
        rostered_df = rostered_df[['Player'] + cols_sorted]
        
        # Optionally, you can sort the DataFrame by current week's ROST
        if f"wk{current_week}" in rostered_df.columns:
            rostered_df = rostered_df.sort_values(by=f"wk{current_week}", ascending=False)
        
        # Store the DataFrame for the position
        rostered_dataframes[position] = rostered_df
        
        # Display the first few rows of the DataFrame for verification
        display(rostered_df.head())
    
    print("Scraping completed for all positions.")
    
    # Return the DataFrames for each position
    return rostered_dataframes

# Scrape rostered percentage over weeks for all positions
positions = ['qb', 'wr', 'rb', 'te', 'flex', 'dst']
rostered_dataframes = scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week')

,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
73,Lamar Jackson(BAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
62,Josh Allen(BUF),99.7,99.7,99.7,99.7,99.7,99.7,99.7
47,Jalen Hurts(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9,97.9
53,Jayden Daniels(WAS),97.8,97.8,97.8,97.8,97.8,97.8,97.8


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
167,Justin Jefferson(MIN),99.8,99.8,99.8,99.8,99.8,99.8,99.8
40,CeeDee Lamb(DAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
10,Amon-Ra St. Brown(DET),99.8,99.8,99.8,99.8,99.8,99.8,99.8
114,Ja'Marr Chase(CIN),99.8,99.8,99.8,99.8,99.8,99.8,99.8
0,A.J. Brown(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
85,Jahmyr Gibbs(DET),99.8,99.8,99.8,99.8,99.8,99.8,99.8
166,Saquon Barkley(PHI),99.8,99.8,99.8,99.8,99.8,99.8,99.8
14,Bijan Robinson(ATL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
105,Jonathan Taylor(IND),99.7,99.7,99.7,99.7,99.7,99.7,99.7
56,Derrick Henry(BAL),99.7,99.7,99.7,99.7,99.7,99.7,99.7


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
172,Travis Kelce(KC),99.5,99.5,99.5,99.5,99.5,99.5,99.5
66,George Kittle(SF),99.1,99.1,99.1,99.1,99.1,99.1,99.1
174,Trey McBride(ARI),98.1,98.1,98.1,98.1,98.1,98.1,98.1
152,Sam LaPorta(DET),97.6,97.6,97.6,97.6,97.6,97.6,97.6
20,Brock Bowers(LV),97.2,97.2,97.2,97.2,97.2,97.2,97.2


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
73,Lamar Jackson(BAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
62,Josh Allen(BUF),99.7,99.7,99.7,99.7,99.7,99.7,99.7
47,Jalen Hurts(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9,97.9
53,Jayden Daniels(WAS),97.8,97.8,97.8,97.8,97.8,97.8,97.8


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
27,San Francisco 49ers(SF),91.8,91.8,91.8,91.8,91.8,91.8,91.8
26,Pittsburgh Steelers(PIT),91.7,91.7,91.7,91.7,91.7,91.7,91.7
24,New York Jets(NYJ),90.3,90.3,90.3,90.3,90.3,90.3,90.3
3,Buffalo Bills(BUF),83.7,83.7,83.7,83.7,83.7,83.7,83.7
9,Denver Broncos(DEN),77.7,77.7,77.7,77.7,77.7,77.7,77.7


Scraping completed for all positions.


In [16]:
# Define the merge_with_rostered_data function
def merge_with_rostered_data(projections_df, rostered_df):
    """
    Merges the projections + salary DataFrame with the rostered percentage data for each player.
    """

    # Split the 'Player' column in the rostered data into 'firstName' and 'lastName'
    rostered_df[['firstName', 'lastName']] = rostered_df['Player'].str.extract(r'(\w+)\s+(.+)\(')
    
    # Merge the projections + salary DataFrame with the rostered DataFrame on firstName and lastName
    merged_df = pd.merge(projections_df, rostered_df[['firstName', 'lastName', 'wk6', 'wk5', 'wk4', 'wk3', 'wk2', 'wk1']], 
                         on=['firstName', 'lastName'], how='left')

    # Rename the columns for clarity
    column_renames = {
        'This Week': 'This Week Salary',
        'Last Week': 'Last Week Salary',
        'Difference': 'Salary Differential',
        'wk6': 'wk6 %rostered',
        'wk5': 'wk5 %rostered',
        'wk4': 'wk4 %rostered',
        'wk3': 'wk3 %rostered',
        'wk2': 'wk2 %rostered',
        'wk1': 'wk1 %rostered'
    }
    
    merged_df.rename(columns=column_renames, inplace=True)
    
    return merged_df


In [17]:
# Assuming rostered_dataframes is the dictionary containing the rostered data scraped for each position
df_qb_merged_projections_salary_rostered = merge_with_rostered_data(df_qb_merge2_projection_salary, rostered_dataframes['qb'])
df_rb_merged_projections_salary_rostered = merge_with_rostered_data(df_rb_merge2_projection_salary, rostered_dataframes['rb'])
df_wr_merged_projections_salary_rostered = merge_with_rostered_data(df_wr_merge2_projection_salary, rostered_dataframes['wr'])

# Display the first few rows for each position
print("QB Merged Projections, Salary, and Rostered DataFrame:")
display(df_qb_merged_projections_salary_rostered.head())

print("\nRB Merged Projections, Salary, and Rostered DataFrame:")
display(df_rb_merged_projections_salary_rostered.head())

print("\nWR Merged Projections, Salary, and Rostered DataFrame:")
display(df_wr_merged_projections_salary_rostered.head())


QB Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,passing_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,15864,Geno Smith,Geno,Smith,QB,SEA,17.68,17.68,...,1.45,"$7,600","$7,900",-300,59.7,59.7,59.7,59.7,59.7,59.7
1,2024,7,4361741,Brock Purdy,Brock,Purdy,QB,SF,17.96,17.96,...,1.66,"$7,700","$7,800",-100,91.2,91.2,91.2,91.2,91.2,91.2
2,2024,7,14880,Kirk Cousins,Kirk,Cousins,QB,ATL,16.80,16.80,...,1.72,"$7,200","$7,400",-200,71.1,71.1,71.1,71.1,71.1,71.1
3,2024,7,3915511,Joe Burrow,Joe,Burrow,QB,CIN,17.91,17.91,...,1.68,"$8,200","$8,100",+100,96.7,96.7,96.7,96.7,96.7,96.7
4,2024,7,8439,Aaron Rodgers,Aaron,Rodgers,QB,NYJ,15.68,15.68,...,1.53,"$6,900","$7,200",-300,54.2,54.2,54.2,54.2,54.2,54.2



RB Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,reception_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,3043078,Derrick Henry,Derrick,Henry,RB,BAL,15.72,16.95,...,0.05,"$9,100","$9,100",0,99.7,99.7,99.7,99.7,99.7,99.7
1,2024,7,4360569,Jordan Mason,Jordan,Mason,RB,SF,11.21,12.81,...,0.04,"$8,800","$8,900",-100,93.4,93.4,93.4,93.4,93.4,93.4
2,2024,7,3929630,Saquon Barkley,Saquon,Barkley,RB,PHI,14.55,17.34,...,0.14,"$9,000","$9,200",-200,99.8,99.8,99.8,99.8,99.8,99.8
3,2024,7,4047365,Josh Jacobs,Josh,Jacobs,RB,GB,11.02,13.46,...,0.09,"$7,300","$7,600",-300,98.7,98.7,98.7,98.7,98.7,98.7
4,2024,7,4241416,Chuba Hubbard,Chuba,Hubbard,RB,CAR,12.46,15.74,...,0.10,"$7,400","$7,400",0,91.8,91.8,91.8,91.8,91.8,91.8



WR Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,reception_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,4362628,Ja'Marr Chase,Ja'Marr,Chase,WR,CIN,11.50,17.54,...,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,7,4047650,DK Metcalf,DK,Metcalf,WR,SEA,9.94,15.22,...,0.47,"$8,100","$8,300",-200,98.3,98.3,98.3,98.3,98.3,98.3
2,2024,7,4258173,Nico Collins,Nico,Collins,WR,HOU,NaN,NaN,...,NaN,"$4,000","$8,900","-4,900",98.5,98.5,98.5,98.5,98.5,98.5
3,2024,7,4262921,Justin Jefferson,Justin,Jefferson,WR,MIN,12.82,19.54,...,0.61,"$9,400",-,-,99.8,99.8,99.8,99.8,99.8,99.8
4,2024,7,4432773,Brian Thomas Jr.,Brian,Thomas Jr.,WR,JAX,8.14,12.54,...,0.39,NaN,NaN,NaN,87.9,87.9,87.9,87.9,87.9,87.9


In [18]:
# Define the base URLs for each position
base_urls = {
    'qb': 'https://www.fantasypros.com/nfl/red-zone-stats/qb.php?week={week}&range=week',
    'rb': 'https://www.fantasypros.com/nfl/red-zone-stats/rb.php?week={week}&range=week',
    'wr': 'https://www.fantasypros.com/nfl/red-zone-stats/wr.php?week={week}&range=week',
    'te': 'https://www.fantasypros.com/nfl/red-zone-stats/te.php?week={week}&range=week'
}


In [19]:
# Function to ensure unique column names
def ensure_unique_column_names(columns):
    col_count = {}
    new_columns = []
    
    for col in columns:
        if col in col_count:
            col_count[col] += 1
            new_columns.append(f"{col}_{col_count[col]}")  # Append suffix to duplicate column
        else:
            col_count[col] = 0
            new_columns.append(col)
    
    return new_columns

# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Ensure unique column names (to handle duplicates)
            week_df.columns = ensure_unique_column_names(week_df.columns)
            
            # Split the 'Player' column into 'FirstName', 'LastName', and 'Team'
            week_df[['firstName', 'lastName', 'team']] = week_df['player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
            
            # Drop the original 'Player' column
            week_df.drop(columns=['player'], inplace=True)
            
            if position == 'qb':
                # Rename columns for passing stats
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'pct': 'pass_pct',
                    'yds': 'pass_yds',
                    'y/a': 'pass_y/a',
                    'td': 'pass_td'
                })
                # Drop specific QB columns by name
                week_df.drop(columns=['pass_att_1', 'pass_yds_1', 'pass_td_1', 'pass_int', 'pass_sacks'], errors='ignore', inplace=True)
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'pass_comp', 'pass_att', 'pass_pct', 'pass_yds', 'pass_y/a', 'pass_td']
            
            elif position == 'rb':
                # Rename rushing and receiving columns
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'y/a': 'rush_y/a',
                    'td': 'rush_td',
                    'pct': 'rush_pct',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds_1': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td_1': 'rec_td',
                    'tgt pct': 'tgt_pct'
                })
                # Drop specific RB columns by name
                week_df.drop(columns=['rush_yds_1', 'rush_td_1'], errors='ignore', inplace=True)
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rush_att', 'rush_yds', 'rush_y/a', 'rush_td', 'rush_pct', 
                                    'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'tgt_pct']
            
            elif position in ['wr', 'te']:
                # Rename receiving columns and drop rushing stats
                week_df = week_df.rename(columns={
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td': 'rec_td',
                    'tgt pct': 'rec_tgt_pct'
                })
                # Drop specific WR and TE columns by name
                week_df.drop(columns=['rec_yds_1', 'rec_td_1'], errors='ignore', inplace=True)
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'rec_tgt_pct']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame

            # Add 'red_zone_' prefix to all columns except the first four columns
            week_df.columns = ['firstName', 'lastName', 'team', 'week'] + [f"red_zone_{col}" for col in week_df.columns[4:]]
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,firstName,lastName,team,week,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td
0,NaN,NaN,NaN,1,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,60.0%,29,5.8,2
2,Baker,Mayfield,TB,1,4,5,80.0%,26,5.2,3
3,Jayden,Daniels,WAS,1,1,1,100.0%,10,10.0,0
4,Derek,Carr,NO,1,2,2,100.0%,19,9.5,2



All weeks data for RB:


,firstName,lastName,team,week,red_zone_rush_att,red_zone_rush_yds,red_zone_rush_y/a,red_zone_rush_td,red_zone_rush_pct,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None,None,None
1,Saquon,Barkley,PHI,1,8,33,4.1,2,100.0%,1,1,100.0%,18,18.0,1,100.0%
2,Jordan,Mason,SF,1,5,26,5.2,1,100.0%,0,0,0%,0,0,0,0%
3,Kyren,Williams,LAR,1,4,8,2.0,1,100.0%,1,1,100.0%,8,8.0,0,100.0%
4,Joe,Mixon,HOU,1,5,15,3.0,1,83.3%,0,0,0%,0,0,0,0%



All weeks data for WR:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None
1,Mike,Evans,TB,1,2,3,66.7%,18,9.0,2,75.0%
2,Stefon,Diggs,HOU,1,3,3,100.0%,11,3.7,2,75.0%
3,NaN,NaN,NaN,1,1,1,100.0%,14,14.0,1,100.0%
4,Cooper,Kupp,LAR,1,2,3,66.7%,13,6.5,1,60.0%



All weeks data for TE:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None
1,Chig,Okonkwo,TEN,1,1,1,100.0%,17,17.0,1,100.0%
2,Juwan,Johnson,NO,1,1,1,100.0%,16,16.0,1,50.0%
3,Kyle,Pitts,ATL,1,1,1,100.0%,12,12.0,1,100.0%
4,Foster,Moreau,NO,1,1,1,100.0%,3,3.0,1,50.0%


In [20]:
# Assign the variables outside the function
df_qb_redzone_all_weeks = position_data['qb']
df_qb_redzone_current_week = current_week_data['qb']

df_rb_redzone_all_weeks = position_data['rb']
df_rb_redzone_current_week = current_week_data['rb']

df_wr_redzone_all_weeks = position_data['wr']
df_wr_redzone_current_week = current_week_data['wr']

df_te_redzone_all_weeks = position_data['te']
df_te_redzone_current_week = current_week_data['te']

# Display QB red zone data for all weeks
print("QB Red Zone Data for All Weeks:")
display(df_qb_redzone_all_weeks.head())

# Display QB red zone data for the current week
print("\nQB Red Zone Data for Current Week:")
display(df_qb_redzone_current_week.head())

QB Red Zone Data for All Weeks:


,firstName,lastName,team,week,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td
0,NaN,NaN,NaN,1,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,60.0%,29,5.8,2
2,Baker,Mayfield,TB,1,4,5,80.0%,26,5.2,3
3,Jayden,Daniels,WAS,1,1,1,100.0%,10,10.0,0
4,Derek,Carr,NO,1,2,2,100.0%,19,9.5,2



QB Red Zone Data for Current Week:


,firstName,lastName,team,week,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td
197,NaN,NaN,NaN,7,None,None,None,None,None,None
198,Russell,Wilson,PIT,7,2,7,28.6%,15,2.1,2
199,Brock,Purdy,SF,7,4,6,66.7%,34,5.7,0
200,Jalen,Hurts,PHI,7,0,1,0.0%,0,0.0,0
201,Jordan,Love,GB,7,5,5,100.0%,36,7.2,2


In [21]:
# Function to optionally save the red zone data to CSV files in the current working directory
def save_redzone_data_to_csv(save_to_csv=False):
    if save_to_csv:
        # Get the current working directory
        current_directory = os.getcwd()

        # Save QB red zone data to CSV in the current directory
        df_qb_redzone_all_weeks.to_csv(os.path.join(current_directory, 'df_qb_redzone_all_weeks.csv'), index=False)
        df_qb_redzone_current_week.to_csv(os.path.join(current_directory, 'df_qb_redzone_current_week.csv'), index=False)
        
        # Save RB red zone data to CSV in the current directory
        df_rb_redzone_all_weeks.to_csv(os.path.join(current_directory, 'df_rb_redzone_all_weeks.csv'), index=False)
        df_rb_redzone_current_week.to_csv(os.path.join(current_directory, 'df_rb_redzone_current_week.csv'), index=False)
        
        # Save WR red zone data to CSV in the current directory
        df_wr_redzone_all_weeks.to_csv(os.path.join(current_directory, 'df_wr_redzone_all_weeks.csv'), index=False)
        df_wr_redzone_current_week.to_csv(os.path.join(current_directory, 'df_wr_redzone_current_week.csv'), index=False)
        
        # Save TE red zone data to CSV in the current directory
        df_te_redzone_all_weeks.to_csv(os.path.join(current_directory, 'df_te_redzone_all_weeks.csv'), index=False)
        df_te_redzone_current_week.to_csv(os.path.join(current_directory, 'df_te_redzone_current_week.csv'), index=False)
        
        print(f"CSV files have been saved in the current directory: {current_directory}")

save_redzone_data_to_csv(save_to_csv=True)

CSV files have been saved in the current directory: /home/rashawn/Desktop/projects/jupyter


In [22]:
# URLs for each position (cumulative redzone stats)
urls_cumulative = {
    'qb': 'https://www.fantasypros.com/nfl/red-zone-stats/qb.php',
    'rb': 'https://www.fantasypros.com/nfl/red-zone-stats/rb.php?range=full',
    'wr': 'https://www.fantasypros.com/nfl/red-zone-stats/wr.php?range=full',
    'te': 'https://www.fantasypros.com/nfl/red-zone-stats/te.php?range=full'
}

In [23]:
# Function to scrape and adjust redzone data for cumulative stats
def scrape_redzone_stats(position, url):
    # Send request to the URL
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for {position}. Status code: {response.status_code}")
        return None
    
    # Parse the content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    
    if not table:
        print(f"No table found for {position}.")
        return None
    
    # Extract table headers
    headers = [th.text.strip() for th in table.find_all('th')]
    
    # Extract rows
    rows = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cols = [td.text.strip() for td in row.find_all('td')]
        rows.append(cols)
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
    
    # Clean up column names and data as necessary
    df.columns = df.columns.str.lower().str.replace(' ', '_')  # Clean column names
    
    # Ensure unique column names (to handle duplicates)
    df.columns = ensure_unique_column_names(df.columns)
    
    # Split 'Player' column into 'firstName', 'lastName', and 'team'
    player_info = df['player'].str.extract(r'(?P<firstName>\w+)\s+(?P<lastName>\w+)\s*\((?P<team>.*)\)')
    df = pd.concat([df, player_info], axis=1)
    df.drop(columns=['player'], inplace=True)  # Drop the original 'player' column
    
    # Handle position-specific logic for renaming and dropping duplicates
    if position == 'qb':
        df = df.rename(columns={
            'comp': 'red_zone_pass_comp',
            'att': 'red_zone_pass_att', 
            'pct': 'red_zone_pass_pct',
            'yds': 'red_zone_pass_yds',
            'y/a': 'red_zone_pass_y/a',
            'td': 'red_zone_pass_td'
        })
        df.drop(columns=['att_dup_1', 'yds_dup_1', 'td_dup_1', 'int', 'sacks'], errors='ignore', inplace=True)
        selected_columns = ['firstName', 'lastName', 'team', 'red_zone_pass_comp', 'red_zone_pass_att', 'red_zone_pass_pct', 'red_zone_pass_yds', 'red_zone_pass_y/a', 'red_zone_pass_td']
    
    elif position == 'rb':
        # Rename rushing and receiving columns
        df = df.rename(columns={
            'att': 'red_zone_rush_att',
            'yds': 'red_zone_rush_yds',
            'y/a': 'red_zone_rush_y/a',
            'td': 'red_zone_rush_td',
            'pct': 'red_zone_rush_pct',
            'rec': 'red_zone_rec_rec',
            'tgt': 'red_zone_rec_tgt',
            'rec_pct': 'red_zone_rec_pct',
            'yds_1': 'red_zone_rec_yds',   # Correct renaming
            'y/r': 'red_zone_rec_y/r',
            'td_1': 'red_zone_rec_td',     # Correct renaming
            'tgt_pct': 'red_zone_tgt_pct'
        })
        # Select and reorder columns
        selected_columns = ['firstName', 'lastName', 'team', 'red_zone_rush_att', 'red_zone_rush_yds', 'red_zone_rush_y/a', 'red_zone_rush_td', 'red_zone_rush_pct', 
                            'red_zone_rec_rec', 'red_zone_rec_tgt', 'red_zone_rec_pct', 'red_zone_rec_yds', 'red_zone_rec_y/r', 'red_zone_rec_td', 'red_zone_tgt_pct']
    
    elif position in ['wr', 'te']:
        df = df.rename(columns={
            'rec': 'red_zone_rec_rec',
            'tgt': 'red_zone_rec_tgt',
            'rec_pct': 'red_zone_rec_pct',
            'yds': 'red_zone_rec_yds',
            'y/r': 'red_zone_rec_y/r',
            'td': 'red_zone_rec_td',
            'tgt_pct': 'red_zone_rec_tgt_pct'
        })
        df.drop(columns=['yds_dup_1', 'td_dup_1'], errors='ignore', inplace=True)
        selected_columns = ['firstName', 'lastName', 'team', 'red_zone_rec_rec', 'red_zone_rec_tgt', 'red_zone_rec_pct', 'red_zone_rec_yds', 'red_zone_rec_y/r', 'red_zone_rec_td', 'red_zone_rec_tgt_pct']
    
    # Filter the DataFrame to the selected columns
    df = df[selected_columns]
    
    return df

# Scrape data for all positions using the updated urls_cumulative dictionary
df_qb_redzone_cumulative = scrape_redzone_stats('qb', urls_cumulative['qb'])
df_rb_redzone_cumulative = scrape_redzone_stats('rb', urls_cumulative['rb'])
df_wr_redzone_cumulative = scrape_redzone_stats('wr', urls_cumulative['wr'])
df_te_redzone_cumulative = scrape_redzone_stats('te', urls_cumulative['te'])

# Display a few rows of each DataFrame to check the results
display(df_qb_redzone_cumulative.head())
display(df_rb_redzone_cumulative.head())
display(df_wr_redzone_cumulative.head())
display(df_te_redzone_cumulative.head())


,firstName,lastName,team,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td
0,NaN,NaN,NaN,None,None,None,None,None,None
1,Josh,Allen,BUF,14,29,48.3%,125,4.3,9
2,Baker,Mayfield,TB,20,26,76.9%,116,4.5,10
3,Lamar,Jackson,BAL,15,25,60.0%,128,5.1,9
4,Justin,Fields,PIT,13,20,65.0%,102,5.1,4


,firstName,lastName,team,red_zone_rush_att,red_zone_rush_yds,red_zone_rush_y/a,red_zone_rush_td,red_zone_rush_pct,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_tgt_pct
0,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None
1,Kyren,Williams,LAR,31,93,3.0,8,88.6%,5,6,83.3%,33,6.6,1,100.0%
2,Derrick,Henry,BAL,16,30,1.9,6,88.9%,2,2,100.0%,9,4.5,1,28.6%
3,David,Montgomery,DET,17,63,3.7,6,48.6%,1,1,100.0%,3,3.0,0,14.3%
4,Alvin,Kamara,NO,20,55,2.8,6,74.1%,1,3,33.3%,1,1.0,0,75.0%


,firstName,lastName,team,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,None,None,None,None,None,None,None
1,Drake,London,ATL,10,12,83.3%,83,8.3,5,63.2%
2,Stefon,Diggs,HOU,5,5,100.0%,34,6.8,3,27.8%
3,NaN,NaN,NaN,8,9,88.9%,57,7.1,3,56.3%
4,Mike,Evans,TB,4,7,57.1%,22,5.5,4,43.8%


,firstName,lastName,team,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,None,None,None,None,None,None,None
1,George,Kittle,SF,11,12,91.7%,74,6.7,5,100.0%
2,Tucker,Kraft,GB,5,6,83.3%,43,8.6,3,100.0%
3,Pat,Freiermuth,PIT,4,4,100.0%,27,6.8,2,66.7%
4,Cole,Kmet,CHI,5,5,100.0%,24,4.8,2,83.3%


In [24]:
# Merge Projections, Salary, Rostered with Redzone data for all positions
def merge_projections_salary_rostered_with_redzone(df_projections_salary_rostered, df_redzone, projection_columns):
    """
    This function merges the projections, salary, and rostered DataFrame with the redzone DataFrame 
    for each position, ensuring the column order and renaming necessary columns.
    """
    # Merge the two dataframes on 'firstName' and 'lastName'
    merged_df = pd.merge(df_projections_salary_rostered, df_redzone, on=['firstName', 'lastName'], how='left')

    # Rename 'team_x' to 'team' (if it exists) and drop 'team_y'
    if 'team_x' in merged_df.columns:
        merged_df.rename(columns={'team_x': 'team'}, inplace=True)
    if 'team_y' in merged_df.columns:
        merged_df.drop(columns=['team_y'], inplace=True)

    # Ensure 'team' column exists
    if 'team' not in merged_df.columns:
        raise KeyError("'team' column is missing from the merged dataframe.")

    # Add a prefix 'projected_' to all projection columns
    projection_columns_with_prefix = {col: f"projected_{col}" for col in projection_columns}
    merged_df.rename(columns=projection_columns_with_prefix, inplace=True)

    return merged_df


# Projection columns for QB
qb_projection_columns = [
    'points', 'points_ppr', 'points_half', 'passing_tds', 'passing_attempts', 'passing_completions', 'passing_yards', 'passing_tds'
]

# Projection columns for RB
rb_projection_columns = [
    'points', 'points_ppr', 'points_half', 'rushing_attempts', 'rushing_yards', 'rushing_tds', 'receptions', 'reception_yards', 'reception_tds'
]

# Projection columns for WR
wr_projection_columns = [
    'points', 'points_ppr', 'points_half', 'receptions', 'reception_yards', 'reception_tds'
]


# Desired column order for QB
qb_columns_order = [
    'year', 'week', 'player_id', 'player', 'firstName', 'lastName', 'position', 'team', 
    'This Week Salary', 'Last Week Salary', 'Salary Differential', 
    'red_zone_pass_comp', 'red_zone_pass_att', 'red_zone_pass_pct', 'red_zone_pass_yds', 'red_zone_pass_y/a', 'red_zone_pass_td',
    'projected_points', 'projected_points_ppr', 'projected_points_half', 'projected_passing_tds', 
    'projected_passing_attempts', 'projected_passing_completions', 'projected_passing_yards', 
    'wk6 %rostered', 'wk5 %rostered', 'wk4 %rostered', 'wk3 %rostered', 'wk2 %rostered', 'wk1 %rostered'
]

# Desired column order for RB
rb_columns_order = [
    'year', 'week', 'player_id', 'player', 'firstName', 'lastName', 'position', 'team', 
    'This Week Salary', 'Last Week Salary', 'Salary Differential', 
    'red_zone_rush_att', 'red_zone_rush_yds', 'red_zone_rush_y/a', 'red_zone_rush_td', 'red_zone_rush_pct', 
    'red_zone_rec_rec', 'red_zone_rec_tgt', 'red_zone_rec_pct', 'red_zone_rec_yds', 'red_zone_rec_y/r', 'red_zone_rec_td', 'red_zone_tgt_pct',
    'projected_points', 'projected_points_ppr', 'projected_points_half', 
    'projected_rushing_attempts', 'projected_rushing_yards', 'projected_rushing_tds', 
    'projected_receptions', 'projected_reception_yards', 'projected_reception_tds', 
    'wk6 %rostered', 'wk5 %rostered', 'wk4 %rostered', 'wk3 %rostered', 'wk2 %rostered', 'wk1 %rostered'
]

# Desired column order for WR
wr_columns_order = [
    'year', 'week', 'player_id', 'player', 'firstName', 'lastName', 'position', 'team', 
    'This Week Salary', 'Last Week Salary', 'Salary Differential', 
    'red_zone_rec_rec', 'red_zone_rec_tgt', 'red_zone_rec_pct', 'red_zone_rec_yds', 'red_zone_rec_y/r', 'red_zone_rec_td', 'red_zone_rec_tgt_pct', 
    'projected_points', 'projected_points_ppr', 'projected_points_half', 
    'projected_receptions', 'projected_reception_yards', 'projected_reception_tds', 
    'wk6 %rostered', 'wk5 %rostered', 'wk4 %rostered', 'wk3 %rostered', 'wk2 %rostered', 'wk1 %rostered'
]


# Merge projections, salary, and rostered data with redzone data
df_qb_projections_salary_rostered_redzone = merge_projections_salary_rostered_with_redzone(df_qb_merged_projections_salary_rostered, df_qb_redzone_cumulative, qb_projection_columns)
df_rb_projections_salary_rostered_redzone = merge_projections_salary_rostered_with_redzone(df_rb_merged_projections_salary_rostered, df_rb_redzone_cumulative, rb_projection_columns)
df_wr_projections_salary_rostered_redzone = merge_projections_salary_rostered_with_redzone(df_wr_merged_projections_salary_rostered, df_wr_redzone_cumulative, wr_projection_columns)

# Reorder columns for QB
df_qb_projections_salary_rostered_redzone = df_qb_projections_salary_rostered_redzone[qb_columns_order]

# Reorder columns for RB
df_rb_projections_salary_rostered_redzone = df_rb_projections_salary_rostered_redzone[rb_columns_order]

# Reorder columns for WR
df_wr_projections_salary_rostered_redzone = df_wr_projections_salary_rostered_redzone[wr_columns_order]

# Save the dataframes to CSV files after reordering
df_qb_projections_salary_rostered_redzone.to_csv('df_qb_projections_salary_rostered_redzone.csv', index=False)
df_rb_projections_salary_rostered_redzone.to_csv('df_rb_projections_salary_rostered_redzone.csv', index=False)
df_wr_projections_salary_rostered_redzone.to_csv('df_wr_projections_salary_rostered_redzone.csv', index=False)

# Display success message
print("CSV files with updated projection columns and reordered columns have been saved.")


CSV files with updated projection columns and reordered columns have been saved.
